# Initialization and basic functions definition
This is the part where we import the useful libraries

In [ ]:
import numpy as np
import h5py
import os
import datetime
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
plt.style.use("mime")

In [ ]:
# Importation de TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # Suppress info messages from TensorFlow
import tensorflow as tf
from keras.models import Sequential
# from keras.metrics import MeanSquaredError
from keras.layers.core import Dense, Activation
# from keras import optimizers
tf.keras.backend.set_floatx('float64')

In [ ]:
def getCurrentTimeDate():
    d = datetime.datetime.now()
    return d.strftime("%a %d/%m/%Y %H:%M:%S")

Import and export data from/to HF5 files

In [ ]:
def writeH5(filename, name, datas):
    h5f = h5py.File(filename, 'w')
    h5f.attrs['Date'] = getCurrentTimeDate()
    h5f.attrs['Name'] = name
    h5f.create_dataset('depsp', data=depspl, dtype = 'float64')
    h5f.create_dataset('T',data=Tl, dtype = 'float64')
    print(h5f.attrs['Date'])
    h5fData = h5f.create_group('Data')
    h5fData.attrs['Number'] = len(datas)
    for data in datas:
        name = str(data[0]) + '_' + str(data[1])
        curData = h5fData.create_dataset(name, data=data[2], dtype = 'float64')
        curData.attrs['depsp'] = data[0]
        curData.attrs['T'] = data[1]
    h5f.close()

def readH5(filename):
    h5f = h5py.File(filename,'r')
    print('Datafile date :',h5f.attrs['Date'])
    print('Datafile name :',h5f.attrs['Name'])
    dataH5 = h5f['Data']
    print('Contains',dataH5.attrs['Number'],'data chunks')
    dl = list(dataH5.keys())
    if (len(dl) != dataH5.attrs['Number']):
        print('Size error')
    else:
        print('Size seems Ok')
    datas = []
    for d in dl:
        data = dataH5[d]
        depsp = data.attrs['depsp']
        T = data.attrs['T']
        name = str(depsp) + '_' + str(T)
        datas.append([depsp, T, data[:]])
    h5f.close()   
    return datas

def writeArrayH5(file, name, data):
    with h5py.File(file, 'a') as hf:
        try:
            del hf[name]
        except:
            pass
        hf.create_dataset(name,  data=data)

def readArrayH5(file, name):
    with h5py.File(file, 'r') as hf:
        data = hf[name][:]
    return data

def writeAnnH5(filename):
    with h5py.File(filename, 'w') as hf:
        i = 1
        hf.create_dataset('layers', data = np.array([len(model.layers)]))
        for layer in model.layers:
            hf.create_dataset('w'+str(i), data = layer.get_weights()[0])
            hf.create_dataset('b'+str(i), data = layer.get_weights()[1])
            i = i + 1
        hf.create_dataset('minData', data = minData)
        hf.create_dataset('maxData', data = maxData)
        hf.create_dataset('deps0', data = np.array([deps0]))
        
def readAnnH5(filename):
    with h5py.File(filename, 'r') as hf:
        layers = hf['layers'][:][0]
        w = []
        b = []
        r = []
        for lay in range(layers):
            w.append(hf['w'+str(lay+1)][:].T)
            b.append(hf['b'+str(lay+1)][:][:,np.newaxis])       
        r.append(hf['minData'][:][:,np.newaxis])
        r.append(hf['maxData'][:][:,np.newaxis])
        r.append(hf['deps0'][:][0])
    return w, b, r

Manipulate and extract Data from list of data

In [ ]:
def getDepsparray(datas):
    l = []
    for d in datas:
        l.append(d[0])
    return np.unique(np.array(l))
    
def getTarray(datas):
    l = []
    for d in datas:
        l.append(d[1])
    return np.unique(np.array(l))

Read and Write CVS files

In [ ]:
def writeCSV(name, d):
    n = name + '_' + str(d[0]) + '_' + str(d[1])
    np.savetxt(n + '.csv', d[2], delimiter = ' ', header = str(d[0]) + '\n' + str(d[1]))

def readCSV(nm):
    d = np.loadtxt(nm)
    f = open(nm,'r')
    l1 = f.readline()
    l2 = f.readline()
    f.close()
    e = float(l1.rsplit()[1])
    T = float(l2.rsplit()[1])
    return [e, T, d]
    
def writeCSVS(name, datas):
    for d in datas:
        writeCSV(name, d)

def readCSVS(ns):
    datas = []
    for n in ns:
        datas.append(readCSV(n))
    return datas

To plot Data

In [ ]:
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x
    
def plotDatas(datas, filename=''):
    Ts = getTarray(datas)
    depsps = getDepsparray(datas)
    ndepsp = depsps.shape[0]
    nT = Ts.shape[0]
    
    plt.figure(figsize = sbPlotSize(ndepsp))
    plt.rc('text', usetex = True)
    plt.subplots_adjust(hspace = 0.3)
    xs, ys = sbPlot(ndepsp)
    idx = 1
    for depsp in depsps:
        plt.subplot(xs, ys, idx)
        cl = 0
        for T in Ts:
            for d in datas:
                if (d[0] == depsp and d[1] == T):
                    subdata = d[2]
                    plt.plot(subdata[:,0], subdata[:,1], label=r'$T=' + str(d[1]) + '^{\circ}C$', linewidth = 2, color=colors[cl])
            cl += 1
        plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
        plt.ylabel(r'flow stress $\sigma$ (MPa)', fontsize = 16) # Labels the y axis
        plt.title(r'strain rate $\dot{\varepsilon} = ' + str(depsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
        plt.xlim(subdata[:,0].min(), subdata[:,0].max())
        #plt.ylim(bottom = 0)
        idx += 1
        
    legendLines = []
    cl = 0
    for temp in list(Ts):
        legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
        cl += 1
    
    if (ndepsp % 2):
        plt.legend([create_dummy_line(**l[1]) for l in legendLines], [l[0] for l in legendLines], 
               loc = 'upper center', fontsize = 12, ncols = 6, bbox_to_anchor = (1.0, -0.2), shadow = False)
    else:
        plt.legend([create_dummy_line(**l[1]) for l in legendLines], [l[0] for l in legendLines], 
               loc = 'upper center', fontsize = 12, ncols = 6, bbox_to_anchor = (0.0, -0.2), shadow = False)
        
    if (filename!=''):
        plt.savefig(filename)
    
    plt.show()